In [1]:
# !pip install easydict

In [2]:
import os
import argparse
import time
import yaml
import logging
from collections import OrderedDict
from contextlib import suppress
from datetime import datetime

import torch
import torchvision.utils
from easydict import EasyDict

from effdet import get_efficientdet_config, create_model, unwrap_bench, create_loader, create_dataset, create_evaluator
from effdet.config.model_config import efficientdet_model_param_dict
from effdet.data import resolve_input_config, SkipSubset
from effdet.anchors import Anchors, AnchorLabeler
from timm.models import resume_checkpoint, load_checkpoint
from timm.models.layers import set_layer_config
from timm.utils import *
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler

In [3]:
print(f'number of configs: {len(efficientdet_model_param_dict)}')
list(efficientdet_model_param_dict.keys())

number of configs: 44


['efficientdet_d0',
 'efficientdet_d1',
 'efficientdet_d2',
 'efficientdet_d3',
 'efficientdet_d4',
 'efficientdet_d5',
 'resdet50',
 'cspresdet50',
 'cspresdext50',
 'cspresdext50pan',
 'cspdarkdet53',
 'cspdarkdet53m',
 'mixdet_m',
 'mixdet_l',
 'mobiledetv2_110d',
 'mobiledetv2_120d',
 'mobiledetv3_large',
 'efficientdet_q0',
 'efficientdet_q1',
 'efficientdet_q2',
 'efficientdet_w0',
 'efficientdet_es',
 'efficientdet_em',
 'efficientdet_lite0',
 'tf_efficientdet_d0',
 'tf_efficientdet_d1',
 'tf_efficientdet_d2',
 'tf_efficientdet_d3',
 'tf_efficientdet_d4',
 'tf_efficientdet_d5',
 'tf_efficientdet_d6',
 'tf_efficientdet_d7',
 'tf_efficientdet_d7x',
 'tf_efficientdet_d0_ap',
 'tf_efficientdet_d1_ap',
 'tf_efficientdet_d2_ap',
 'tf_efficientdet_d3_ap',
 'tf_efficientdet_d4_ap',
 'tf_efficientdet_d5_ap',
 'tf_efficientdet_lite0',
 'tf_efficientdet_lite1',
 'tf_efficientdet_lite2',
 'tf_efficientdet_lite3',
 'tf_efficientdet_lite4']

In [4]:
setup_default_logging()
parser = argparse.ArgumentParser(description='Training Config', add_help=False)
with open('./defaults.yaml', 'r') as f:
    cfg = yaml.safe_load(f)
    parser.set_defaults(**cfg)
args = parser.parse_args([])
args_text = yaml.safe_dump(args.__dict__, default_flow_style=False)

In [5]:
model_config = get_efficientdet_config('tf_efficientdet_d0')
model_config.__dict__

{'_metadata': ContainerMetadata(ref_type=typing.Any, object_type=None, optional=True, key=None, flags={}, flags_root=False, resolver_cache=defaultdict(<class 'dict'>, {}), key_type=typing.Any, element_type=typing.Any),
 '_parent': None,
 '_flags_cache': {'struct': None, 'readonly': None, 'allow_objects': None},
 '_content': {'name': 'tf_efficientdet_d0',
  'backbone_name': 'tf_efficientnet_b0',
  'backbone_args': {'drop_path_rate': 0.2},
  'backbone_indices': None,
  'image_size': [512, 512],
  'num_classes': 90,
  'min_level': 3,
  'max_level': 7,
  'num_levels': 5,
  'num_scales': 3,
  'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]],
  'anchor_scale': 4.0,
  'pad_type': 'same',
  'act_type': 'swish',
  'norm_layer': None,
  'norm_kwargs': {'eps': 0.001, 'momentum': 0.01},
  'box_class_repeats': 3,
  'fpn_cell_repeats': 3,
  'fpn_channels': 64,
  'separable_conv': True,
  'apply_resample_bn': True,
  'conv_after_downsample': False,
  'conv_bn_relu_pattern': False,
  'use_native_

In [6]:
args.__dict__.update(model_config)
args.__dict__

{'amp': True,
 'batch_size': 32,
 'bench_labeler': False,
 'channels_last': False,
 'clip_grad': 10.0,
 'cooldown_epochs': 10,
 'dataset': 'fashion',
 'decay_epochs': 30,
 'decay_rate': 0.1,
 'distributed': False,
 'dist_bn': '',
 'epochs': 200,
 'eval_metric': 'map',
 'fill_color': None,
 'initial_checkpoint': '',
 'interpolation': '',
 'jit_loss': False,
 'legacy_focal': False,
 'local_rank': 0,
 'log_interval': 50,
 'lr': 0.01,
 'lr_cycle_limit': 1,
 'lr_cycle_mul': 1.0,
 'lr_noise': None,
 'lr_noise_pct': 0.67,
 'lr_noise_std': 1.0,
 'mean': None,
 'min_lr': 1e-05,
 'model': 'tf_efficientdet_d0',
 'model_ema': True,
 'model_ema_decay': 0.9998,
 'momentum': 0.9,
 'native_amp': True,
 'no_prefetcher': False,
 'no_pretrained_backbone': False,
 'no_resume_opt': False,
 'opt': 'momentum',
 'opt_eps': 0.001,
 'output': './result',
 'patience_epochs': 10,
 'pin_mem': False,
 'pretrained': False,
 'recount': 1,
 'recovery_interval': 0,
 'redundant_bias': True,
 'remode': 'pixel',
 'reprob'

In [7]:
# hyper parameters 변경
args.dataset = 'fashion'
args.root = './dataset/Fashion'
args.output = './output'
args.num_classes = 4
args.image_size = [512, 512]
args.epochs = 200
args.batch_size = 32
args.opt = 'momentum'
args.opt_eps = 0.001
args.momentum = 0.9
args.lr = 0.01
args.weight_decay = 4e-5
args.warmup_epochs = 5
args.warmup_lr = 0.0001
args.cooldown_epochs = 10
args.model_ema = True
args.model_ema_decay = 0.9998
# ...

# init
args.pretrained_backbone = not args.no_pretrained_backbone
args.prefetcher = not args.no_prefetcher
args.device = 'cuda:0'
args.rank = 0  # global rank

In [8]:
def main(args):
    torch.manual_seed(args.seed + args.rank)

    model = create_model(
        args.model,
        bench_task='train',
        num_classes=args.num_classes,
        pretrained=args.pretrained,
        pretrained_backbone=args.pretrained_backbone,
        redundant_bias=args.redundant_bias,
        label_smoothing=args.smoothing,
        legacy_focal=args.legacy_focal,
        jit_loss=args.jit_loss,
        soft_nms=args.soft_nms,
        bench_labeler=args.bench_labeler,
        checkpoint_path=args.initial_checkpoint,
    )
    model_config = model.config

    if args.local_rank == 0:
        logging.info('Model %s created, param count: %d' % (args.model, sum([m.numel() for m in model.parameters()])))

    model.cuda()
    if args.channels_last:
        model = model.to(memory_format=torch.channels_last)

        
    optimizer = create_optimizer(args, model)
    
    
    amp_autocast = suppress
    loss_scaler = None
    if args.amp:
        amp_autocast = torch.cuda.amp.autocast
        loss_scaler = NativeScaler()
        if args.local_rank == 0:
            logging.info('Using native Torch AMP. Training in mixed precision.')
    else:
        if args.local_rank == 0:
            logging.info('AMP not enabled. Training in float32.')


    # optionally resume from a checkpoint
    resume_epoch = None
    if args.resume:
        resume_epoch = resume_checkpoint(
            unwrap_bench(model), args.resume,
            optimizer=None if args.no_resume_opt else optimizer,
            loss_scaler=None if args.no_resume_opt else loss_scaler,
            log_info=args.local_rank == 0)
        
        
    model_ema = None
    if args.model_ema:
        model_ema = ModelEmaV2(model, decay=args.model_ema_decay)
        if args.resume:
            load_checkpoint(unwrap_bench(model_ema), args.resume, use_ema=True)
            
            
    lr_scheduler, num_epochs = create_scheduler(args, optimizer)
    start_epoch = 0
    if args.start_epoch is not None:
        # a specified start_epoch will always override the resume epoch
        start_epoch = args.start_epoch
    elif resume_epoch is not None:
        start_epoch = resume_epoch
    if lr_scheduler is not None and start_epoch > 0:
        lr_scheduler.step(start_epoch)

    if args.local_rank == 0:
        logging.info('Scheduled epochs: {}'.format(num_epochs))
        
        
    loader_train, loader_eval, evaluator = create_datasets_and_loaders(args, model_config)

    if model_config.num_classes < loader_train.dataset.parser.max_label:
        logging.error(
            f'Model {model_config.num_classes} has fewer classes than dataset {loader_train.dataset.parser.max_label}.')
        exit(1)
    if model_config.num_classes > loader_train.dataset.parser.max_label:
        logging.warning(
            f'Model {model_config.num_classes} has more classes than dataset {loader_train.dataset.parser.max_label}.')
        
        
    eval_metric = args.eval_metric
    best_metric = None
    best_epoch = None
    saver = None
    output_dir = ''
    if args.local_rank == 0:
        output_base = args.output if args.output else './output'
        exp_name = '-'.join([
            datetime.now().strftime("%Y%m%d-%H%M%S"),
            args.model
        ])
        output_dir = get_outdir(output_base, 'train', exp_name)
        decreasing = True if eval_metric == 'loss' else False
        saver = CheckpointSaver(
            model, optimizer, args=args, model_ema=model_ema, amp_scaler=loss_scaler,
            checkpoint_dir=output_dir, decreasing=decreasing, unwrap_fn=unwrap_bench)
        with open(os.path.join(output_dir, 'args.yaml'), 'w') as f:
            f.write(args_text)
            
    
    try:
        for epoch in range(start_epoch, num_epochs):
            if args.distributed:
                loader_train.sampler.set_epoch(epoch)

            train_metrics = train_epoch(
                epoch, model, loader_train, optimizer, args,
                lr_scheduler=lr_scheduler, saver=saver, output_dir=output_dir,
                amp_autocast=amp_autocast, loss_scaler=loss_scaler, model_ema=model_ema)

            if args.distributed and args.dist_bn in ('broadcast', 'reduce'):
                if args.local_rank == 0:
                    logging.info("Distributing BatchNorm running means and vars")
                distribute_bn(model, args.world_size, args.dist_bn == 'reduce')

            # the overhead of evaluating with coco style datasets is fairly high, so just ema or non, not both
            if model_ema is not None:
                if args.distributed and args.dist_bn in ('broadcast', 'reduce'):
                    distribute_bn(model_ema, args.world_size, args.dist_bn == 'reduce')

                eval_metrics = validate(model_ema.module, loader_eval, args, evaluator, log_suffix=' (EMA)')
            else:
                eval_metrics = validate(model, loader_eval, args, evaluator)

            if lr_scheduler is not None:
                # step LR for next epoch
                lr_scheduler.step(epoch + 1, eval_metrics[eval_metric])

            if saver is not None:
                update_summary(
                    epoch, train_metrics, eval_metrics, os.path.join(output_dir, 'summary.csv'),
                    write_header=best_metric is None)

                # save proper checkpoint with eval metric
                best_metric, best_epoch = saver.save_checkpoint(epoch=epoch, metric=eval_metrics[eval_metric])

    except KeyboardInterrupt:
        pass
    if best_metric is not None:
        logging.info('*** Best metric: {0} (epoch {1})'.format(best_metric, best_epoch))

In [9]:
def create_datasets_and_loaders(
        args,
        model_config,
        transform_train_fn=None,
        transform_eval_fn=None,
        collate_fn=None,
):
    """ Setup datasets, transforms, loaders, evaluator.
    Args:
        args: Command line args / config for training
        model_config: Model specific configuration dict / struct
        transform_train_fn: Override default image + annotation transforms (see note in loaders.py)
        transform_eval_fn: Override default image + annotation transforms (see note in loaders.py)
        collate_fn: Override default fast collate function
    Returns:
        Train loader, validation loader, evaluator
    """
    input_config = resolve_input_config(args, model_config=model_config)

    dataset_train, dataset_eval = create_dataset(args.dataset, args.root)

    # setup labeler in loader/collate_fn if not enabled in the model bench
    labeler = None
    if not args.bench_labeler:
        labeler = AnchorLabeler(
            Anchors.from_config(model_config), model_config.num_classes, match_threshold=0.5)

    loader_train = create_loader(
        dataset_train,
        input_size=input_config['input_size'],
        batch_size=args.batch_size,
        is_training=True,
        use_prefetcher=args.prefetcher,
        re_prob=args.reprob,
        re_mode=args.remode,
        re_count=args.recount,
        # color_jitter=args.color_jitter,
        # auto_augment=args.aa,
        interpolation=args.train_interpolation or input_config['interpolation'],
        fill_color=input_config['fill_color'],
        mean=input_config['mean'],
        std=input_config['std'],
        num_workers=args.workers,
        distributed=args.distributed,
        pin_mem=args.pin_mem,
        anchor_labeler=labeler,
        transform_fn=transform_train_fn,
        collate_fn=collate_fn,
    )

    if args.val_skip > 1:
        dataset_eval = SkipSubset(dataset_eval, args.val_skip)
    loader_eval = create_loader(
        dataset_eval,
        input_size=input_config['input_size'],
        batch_size=args.batch_size,
        is_training=False,
        use_prefetcher=args.prefetcher,
        interpolation=input_config['interpolation'],
        fill_color=input_config['fill_color'],
        mean=input_config['mean'],
        std=input_config['std'],
        num_workers=args.workers,
        distributed=args.distributed,
        pin_mem=args.pin_mem,
        anchor_labeler=labeler,
        transform_fn=transform_eval_fn,
        collate_fn=collate_fn,
    )

    evaluator = create_evaluator(args.dataset, loader_eval.dataset, distributed=args.distributed, pred_yxyx=False)

    return loader_train, loader_eval, evaluator

In [10]:
def train_epoch(epoch, model, loader, optimizer, args,
                lr_scheduler=None, saver=None, output_dir='', amp_autocast=suppress, loss_scaler=None, model_ema=None):

    batch_time_m = AverageMeter()
    data_time_m = AverageMeter()
    losses_m = AverageMeter()

    model.train()

    end = time.time()
    last_idx = len(loader) - 1
    num_updates = epoch * len(loader)
    for batch_idx, (input, target) in enumerate(loader):
        last_batch = batch_idx == last_idx
        data_time_m.update(time.time() - end)

        if args.channels_last:
            input = input.contiguous(memory_format=torch.channels_last)

        with amp_autocast():
            output = model(input, target)
        loss = output['loss']

        if not args.distributed:
            losses_m.update(loss.item(), input.size(0))

        optimizer.zero_grad()
        if loss_scaler is not None:
            loss_scaler(loss, optimizer, clip_grad=args.clip_grad, parameters=model.parameters())
        else:
            loss.backward()
            if args.clip_grad:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad)
            optimizer.step()

        torch.cuda.synchronize()
        if model_ema is not None:
            model_ema.update(model)
        num_updates += 1

        batch_time_m.update(time.time() - end)
        if last_batch or batch_idx % args.log_interval == 0:
            lrl = [param_group['lr'] for param_group in optimizer.param_groups]
            lr = sum(lrl) / len(lrl)

            if args.distributed:
                reduced_loss = reduce_tensor(loss.data, args.world_size)
                losses_m.update(reduced_loss.item(), input.size(0))

            if args.local_rank == 0:
                logging.info(
                    'Train: {} [{:>4d}/{} ({:>3.0f}%)]  '
                    'Loss: {loss.val:>9.6f} ({loss.avg:>6.4f})  '
                    'LR: {lr:.3e}'.format(
                        epoch,
                        batch_idx, len(loader),
                        100. * batch_idx / last_idx,
                        loss=losses_m,
                        lr=lr))

                if args.save_images and output_dir:
                    torchvision.utils.save_image(
                        input,
                        os.path.join(output_dir, 'train-batch-%d.jpg' % batch_idx),
                        padding=0,
                        normalize=True)

        if saver is not None and args.recovery_interval and (
                last_batch or (batch_idx + 1) % args.recovery_interval == 0):
            saver.save_recovery(epoch, batch_idx=batch_idx)

        if lr_scheduler is not None:
            lr_scheduler.step_update(num_updates=num_updates, metric=losses_m.avg)

        end = time.time()
        # end for

    if hasattr(optimizer, 'sync_lookahead'):
        optimizer.sync_lookahead()

    return OrderedDict([('loss', losses_m.avg)])

In [11]:
def validate(model, loader, args, evaluator=None, log_suffix=''):
    batch_time_m = AverageMeter()
    losses_m = AverageMeter()

    model.eval()

    end = time.time()
    last_idx = len(loader) - 1
    with torch.no_grad():
        for batch_idx, (input, target) in enumerate(loader):
            last_batch = batch_idx == last_idx

            output = model(input, target)
            loss = output['loss']

            if evaluator is not None:
                evaluator.add_predictions(output['detections'], target)

            if args.distributed:
                reduced_loss = reduce_tensor(loss.data, args.world_size)
            else:
                reduced_loss = loss.data

            torch.cuda.synchronize()

            losses_m.update(reduced_loss.item(), input.size(0))

            batch_time_m.update(time.time() - end)
            end = time.time()
            if args.local_rank == 0 and (last_batch or batch_idx % args.log_interval == 0):
                log_name = 'Test' + log_suffix
                logging.info(
                    '{0}: [{1:>4d}/{2}]  '
                    'Time: {batch_time.val:.3f} ({batch_time.avg:.3f})  '
                    'Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  '.format(
                        log_name, batch_idx, last_idx, batch_time=batch_time_m, loss=losses_m))

    metrics = OrderedDict([('loss', losses_m.avg)])
    if evaluator is not None:
        metrics['map'] = evaluator.evaluate()

    return metrics

In [ ]:
main(args)

Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b0_aa-827b6e33.pth)
Model tf_efficientdet_d0 created, param count: 3829757
Using native Torch AMP. Training in mixed precision.
Scheduled epochs: 210


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Train: 0 [   0/305 (  0%)]  Loss: 10.855341 (10.8553)  LR: 1.000e-04
Train: 0 [  50/305 ( 16%)]  Loss:  6.218544 (9.4309)  LR: 1.000e-04
Train: 0 [ 100/305 ( 33%)]  Loss:  6.013713 (7.7762)  LR: 1.000e-04
Train: 0 [ 150/305 ( 49%)]  Loss:  5.166361 (6.9206)  LR: 1.000e-04
Train: 0 [ 200/305 ( 66%)]  Loss:  4.380965 (6.3390)  LR: 1.000e-04
Train: 0 [ 250/305 ( 82%)]  Loss:  3.870913 (5.9198)  LR: 1.000e-04
Train: 0 [ 300/305 ( 99%)]  Loss:  3.540631 (5.5851)  LR: 1.000e-04
Train: 0 [ 304/305 (100%)]  Loss:  3.710784 (5.5627)  LR: 1.000e-04
Test (EMA): [   0/30]  Time: 1.255 (1.255)  Loss:  1.7650 (1.7650)  
Test (EMA): [  30/30]  Time: 0.117 (0.257)  Loss:  1.7699 (1.7705)  


Loading and preparing results...
DONE (t=0.46s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.69s).
Accumulating evaluation results...
DONE (t=0.63s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.002
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.004
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

Current checkpoints:
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-0.pth.tar', 8.12787333623758e-06)

Train: 1 [   0/305 (  0%)]  Loss:  4.040983 (4.0410)  LR: 2.080e-03
Train: 1 [  50/305 ( 16%)]  Loss:  1.899486 (2.3627)  LR: 2.080e-03
Train: 1 [ 100/305 ( 33%)]  Loss:  1.800848 (2.1023)  LR: 2.080e-03
Train: 1 [ 150/305 ( 49%)]  Loss:  1.744509 (1.9893)  LR: 2.080e-03
Train: 1 [ 200/305 ( 66%)]  Loss:  1.724537 (1.9253)  LR: 2.080e-03
Train: 1 [ 250/305 ( 82%)]  Loss:  1.669736 (1.8822)  LR: 2.080e-03
Train: 1 [ 300/305 ( 99%)]  Loss:  1.689896 (1.8494)  LR: 2.080e-03
Train: 1 [ 304/305 (100%)]  Loss:  1.667379 (1.8472)  LR: 2.080e-03
Test (EMA): [   0/30]  Time: 0.908 (0.908)  Loss:  1.7595 (1.7595)  
Test (EMA): [  30/30]  Time: 0.116 (0.232)  Loss:  1.7662 (1.7659)  


...

Current checkpoints:
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-20.pth.tar', 0.019264962195950888)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-19.pth.tar', 0.01552453484862187)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-18.pth.tar', 0.012636855791893484)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-17.pth.tar', 0.010224709921869704)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-16.pth.tar', 0.008653910288775067)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-15.pth.tar', 0.0068597551615406736)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-14.pth.tar', 0.005564464504363815)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-13.pth.tar', 0.00454191857464401)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-12.pth.tar', 0.00382481359329111)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-11.pth.tar', 0.00319456545

Loading and preparing results...
DONE (t=0.54s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.67s).
Accumulating evaluation results...
DONE (t=0.82s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.025
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.096
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.026
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.095
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.235
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.267
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

Current checkpoints:
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-21.pth.tar', 0.025427845246064437)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-20.pth.tar', 0.019264962195950888)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-19.pth.tar', 0.01552453484862187)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-18.pth.tar', 0.012636855791893484)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-17.pth.tar', 0.010224709921869704)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-16.pth.tar', 0.008653910288775067)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-15.pth.tar', 0.0068597551615406736)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-14.pth.tar', 0.005564464504363815)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-13.pth.tar', 0.00454191857464401)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-12.pth.tar', 0.0038248135

Loading and preparing results...
DONE (t=0.54s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.65s).
Accumulating evaluation results...
DONE (t=0.68s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.116
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.006
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.032
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.118
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.263
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.290
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

Current checkpoints:
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-22.pth.tar', 0.031765278932805834)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-21.pth.tar', 0.025427845246064437)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-20.pth.tar', 0.019264962195950888)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-19.pth.tar', 0.01552453484862187)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-18.pth.tar', 0.012636855791893484)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-17.pth.tar', 0.010224709921869704)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-16.pth.tar', 0.008653910288775067)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-15.pth.tar', 0.0068597551615406736)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-14.pth.tar', 0.005564464504363815)
 ('./output/train/20211023-181844-tf_efficientdet_d0/checkpoint-13.pth.tar', 0.004541918